# Face Metrics Experiments

In [1]:
import sys
sys.path.append("..")

from newMesh import FaceTetrahedronMesh
from PolyllaFace import PolyllaFace
from utils import vtk_to_obj
import time
import ctypes
import pathlib

polylla_inradoius = []
polylla_circumradoius = []
polylla_area = []

inradious = 'r'
circumradious = 'R'
area = 'a'

n_experiments = 5

In [2]:
def run_polylla(n, distribution, flag):
    filename = "../data/"+ str(n) + distribution +".1"
    node_file = filename + ".node"
    ele_file = filename + ".ele"
    face_file = filename + ".face"

    t0 = time.time()
    mesh= FaceTetrahedronMesh(node_file, face_file, ele_file)
    polyllaFace_mesh = PolyllaFace(mesh,flag)
    tf = time.time()

    dtM = tf - t0 

    print('Tiempo:',dtM,'segs')
    edge_ratio = polyllaFace_mesh.edge_ratio()
    tetra_per_poly = polyllaFace_mesh.tetra_per_poly()
    faces_per_poly = polyllaFace_mesh.faces_per_poly()
    print('\n Stats polylla:')
    polyllaFace_mesh.get_info()
    print('Mean Edge Ratio:', edge_ratio[0])
    print('Minimum Edge Ratio:', edge_ratio[1])
    print('Maximum Edge Ratio:', edge_ratio[2])
    print('Mean Tetrahedron per Polyhedron:', tetra_per_poly[0])
    print('Minimum Tetrahedron per Polyhedron:', tetra_per_poly[1])
    print('Maximum Tetrahedron per Polyhedron:', tetra_per_poly[2])
    print('Mean Faces per Polyhedron:', faces_per_poly[0])
    print('Minimum Faces per Polyhedron:', faces_per_poly[1])
    print('Maximum Faces per Polyhedron:', faces_per_poly[2])

    print()
    return polyllaFace_mesh

## Random Distribution

In [3]:
distribution = 'random'

#### n = 100

In [4]:
n = 100

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 0.17785143852233887 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 128
Number of barrier faces: 31
Number of polyhedra with barrier faces: 23
Number of polyhedrons that are tetrahedrons: 24
Mean Edge Ratio: 0.0921567263108631
Minimum Edge Ratio: 0.0018669354440557005
Maximum Edge Ratio: 0.38414427673923235
Mean Tetrahedron per Polyhedron: 3.4453125
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 14
Mean Faces per Polyhedron: 8.890625
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 30

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 0.21106338500976562 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 144
Number of barrier faces: 101
Number of polyhedra with barrier faces: 34
Number of polyhedrons that are tetrahedrons: 32
Mean Edge

#### n = 500

In [5]:
n = 500

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 4.592802286148071 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 726
Number of barrier faces: 209
Number of polyhedra with barrier faces: 145
Number of polyhedrons that are tetrahedrons: 136
Mean Edge Ratio: 0.098309648651974
Minimum Edge Ratio: 2.4934446865405707e-05
Maximum Edge Ratio: 0.5096617549864849
Mean Tetrahedron per Polyhedron: 3.275482093663912
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 18
Mean Faces per Polyhedron: 8.550964187327823
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 38

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 5.499751806259155 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 734
Number of barrier faces: 549
Number of polyhedra with barrier faces: 179
Number of polyhedrons that are tetrahedro

#### n = 1000

In [6]:
n = 1000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 18.04891324043274 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 1407
Number of barrier faces: 421
Number of polyhedra with barrier faces: 284
Number of polyhedrons that are tetrahedrons: 268
Mean Edge Ratio: 0.08782518826286619
Minimum Edge Ratio: 5.939612159773522e-07
Maximum Edge Ratio: 0.48696115748832136
Mean Tetrahedron per Polyhedron: 3.4371002132196162
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 20
Mean Faces per Polyhedron: 8.874200426439232
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 42

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 18.18607783317566 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 1421
Number of barrier faces: 1173
Number of polyhedra with barrier faces: 329
Number of polyhedrons that are tetr

In [4]:
n = 1000
filename = "../data/"+ str(n) + distribution +".1"
vtk_to_obj(filename)

2024-01-11 10:35:15.575 ( 146.156s) [        26C19740]      vtkDataReader.cxx:522    ERR| vtkUnstructuredGridReader (0x43e4c40): Unable to open file: ../data/1000random.1.vtk.vtk


AttributeError: 'NoneType' object has no attribute 'GetNumberOfPoints'

##### n = 5000

In [7]:
n = 5000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 435.1147515773773 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 6990
Number of barrier faces: 1975
Number of polyhedra with barrier faces: 1299
Number of polyhedrons that are tetrahedrons: 1596
Mean Edge Ratio: 0.07287930039138193
Minimum Edge Ratio: 5.7472427817186655e-09
Maximum Edge Ratio: 0.452153945188356
Mean Tetrahedron per Polyhedron: 3.390844062947067
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 21
Mean Faces per Polyhedron: 8.781688125894135
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 44

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 415.1286597251892 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 6680
Number of barrier faces: 5443
Number of polyhedra with barrier faces: 1459
Number of polyhedrons that are te

#### n = 10000

In [8]:
n = 10000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns


Tiempo: 1620.2229204177856 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 14440
Number of barrier faces: 3874
Number of polyhedra with barrier faces: 2622
Number of polyhedrons that are tetrahedrons: 3534
Mean Edge Ratio: 0.0708559479175555
Minimum Edge Ratio: 2.8162128837440787e-10
Maximum Edge Ratio: 0.5256758465615928
Mean Tetrahedron per Polyhedron: 3.280747922437673
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 25
Mean Faces per Polyhedron: 8.561495844875346
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 52

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 1654.8153126239777 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 13348
Number of barrier faces: 10665
Number of polyhedra with barrier faces: 2932
Number of polyhedrons that are tetrahedrons: 3368
Mean Edge Ratio: 0.08059157477611437
Minimum Edge Ratio: 2.8162128837440787e-10
Maximum E

## Poisson Distribution

In [9]:
distribution = 'poisson'

#### n = 100

In [10]:
n = 100

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 4.723656415939331 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 798
Number of barrier faces: 200
Number of polyhedra with barrier faces: 153
Number of polyhedrons that are tetrahedrons: 147
Mean Edge Ratio: 0.25296107506391546
Minimum Edge Ratio: 0.016140368797698593
Maximum Edge Ratio: 0.5865062280322514
Mean Tetrahedron per Polyhedron: 3.112781954887218
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 12
Mean Faces per Polyhedron: 8.225563909774436
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 26

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns


Tiempo: 4.909750938415527 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 880
Number of barrier faces: 479
Number of polyhedra with barrier faces: 223
Number of polyhedrons that are tetrahedrons: 256
Mean Edge Ratio: 0.2699573975402832
Minimum Edge Ratio: 0.033263406050753085
Maximum Edge Ratio: 0.6273021374567038
Mean Tetrahedron per Polyhedron: 2.8215909090909093
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 11
Mean Faces per Polyhedron: 7.6431818181818185
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 24

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 4.884063720703125 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 842
Number of barrier faces: 379
Number of polyhedra with barrier faces: 209
Number of polyhedrons that are tetrahedrons: 222
Mean Edge Ratio: 1.0
Minimum Edge Ratio: 1.0
Maximum Edge Ratio: 1.0
Mean Tetrahedron per Polyhedron: 

#### n = 500

In [11]:
n = 500

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns


Tiempo: 4.703016042709351 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 774
Number of barrier faces: 216
Number of polyhedra with barrier faces: 157
Number of polyhedrons that are tetrahedrons: 127
Mean Edge Ratio: 0.2549823219844767
Minimum Edge Ratio: 7.647544771746064e-05
Maximum Edge Ratio: 0.5619177012352183
Mean Tetrahedron per Polyhedron: 3.1847545219638245
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 13
Mean Faces per Polyhedron: 8.369509043927648
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 28

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 4.875194072723389 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 875
Number of barrier faces: 472
Number of polyhedra with barrier faces: 222
Number of polyhedrons that are tetrahedrons: 241
Mean Edge Ratio: 0.2709477068769925
Minimum Edge Ratio: 7.647544771746064e-05
Maximum Edge Ratio: 0.61

#### n = 1000

In [12]:
n = 1000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns


Tiempo: 4.455381393432617 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 790
Number of barrier faces: 205
Number of polyhedra with barrier faces: 153
Number of polyhedrons that are tetrahedrons: 152
Mean Edge Ratio: 0.2491576944029566
Minimum Edge Ratio: 0.004818052786646338
Maximum Edge Ratio: 0.5916734300638071
Mean Tetrahedron per Polyhedron: 3.0443037974683542
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 13
Mean Faces per Polyhedron: 8.08860759493671
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 28

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 4.623171091079712 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 848
Number of barrier faces: 462
Number of polyhedra with barrier faces: 207
Number of polyhedrons that are tetrahedrons: 243
Mean Edge Ratio: 0.2634203451468116
Minimum Edge Ratio: 0.0047608668645256385
Maximum Edge Ratio: 0.5926

##### n = 5000

In [13]:
n = 5000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns


Tiempo: 4.548574924468994 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 782
Number of barrier faces: 209
Number of polyhedra with barrier faces: 150
Number of polyhedrons that are tetrahedrons: 136
Mean Edge Ratio: 0.24639963317288865
Minimum Edge Ratio: 0.0057801751149192775
Maximum Edge Ratio: 0.6129189204628973
Mean Tetrahedron per Polyhedron: 3.0920716112531967
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 13
Mean Faces per Polyhedron: 8.184143222506394
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 28

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 4.654970645904541 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 830
Number of barrier faces: 459
Number of polyhedra with barrier faces: 204
Number of polyhedrons that are tetrahedrons: 209
Mean Edge Ratio: 0.25885872862133613
Minimum Edge Ratio: 0.0057801751149192775
Maximum Edge Ratio: 0.

#### n = 10000

In [14]:
n = 10000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns


Tiempo: 4.503384113311768 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 745
Number of barrier faces: 185
Number of polyhedra with barrier faces: 136
Number of polyhedrons that are tetrahedrons: 124
Mean Edge Ratio: 0.2561094474582076
Minimum Edge Ratio: 0.009591936163852031
Maximum Edge Ratio: 0.5340259938083121
Mean Tetrahedron per Polyhedron: 3.2362416107382552
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 13
Mean Faces per Polyhedron: 8.47248322147651
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 28

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 4.58025598526001 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 817
Number of barrier faces: 453
Number of polyhedra with barrier faces: 214
Number of polyhedrons that are tetrahedrons: 228
Mean Edge Ratio: 0.27365302528438207
Minimum Edge Ratio: 0.009591936163852031
Maximum Edge Ratio: 0.64703

## Semiuniform Distribution

In [15]:
distribution = 'semiuniform'

#### n = 100

In [16]:
n = 100

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 5.105965614318848 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 711
Number of barrier faces: 198
Number of polyhedra with barrier faces: 137
Number of polyhedrons that are tetrahedrons: 129
Mean Edge Ratio: 0.33616843588457035
Minimum Edge Ratio: 0.09759307171854792
Maximum Edge Ratio: 0.6564894374861545
Mean Tetrahedron per Polyhedron: 3.1322081575246132
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 12
Mean Faces per Polyhedron: 8.264416315049226
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 26

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns


Tiempo: 4.0178608894348145 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 776
Number of barrier faces: 350
Number of polyhedra with barrier faces: 196
Number of polyhedrons that are tetrahedrons: 182
Mean Edge Ratio: 0.3529691116042896
Minimum Edge Ratio: 0.08437592784170189
Maximum Edge Ratio: 0.7214320548559369
Mean Tetrahedron per Polyhedron: 2.8698453608247423
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 12
Mean Faces per Polyhedron: 7.739690721649485
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 26

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 4.0084474086761475 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 748
Number of barrier faces: 301
Number of polyhedra with barrier faces: 178
Number of polyhedrons that are tetrahedrons: 155
Mean Edge Ratio: 1.0
Minimum Edge Ratio: 1.0
Maximum Edge Ratio: 1.0
Mean Tetrahedron per Polyhedron: 

#### n = 500

In [17]:
n = 500

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns


Tiempo: 3.8935546875 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 711
Number of barrier faces: 198
Number of polyhedra with barrier faces: 137
Number of polyhedrons that are tetrahedrons: 129
Mean Edge Ratio: 0.33616843588457035
Minimum Edge Ratio: 0.09759307171854792
Maximum Edge Ratio: 0.6564894374861545
Mean Tetrahedron per Polyhedron: 3.1322081575246132
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 12
Mean Faces per Polyhedron: 8.264416315049226
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 26

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 4.0172483921051025 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 776
Number of barrier faces: 350
Number of polyhedra with barrier faces: 196
Number of polyhedrons that are tetrahedrons: 182
Mean Edge Ratio: 0.3529691116042896
Minimum Edge Ratio: 0.08437592784170189
Maximum Edge Ratio: 0.721432054

#### n = 1000

In [18]:
n = 1000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns


Tiempo: 3.899639368057251 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 711
Number of barrier faces: 198
Number of polyhedra with barrier faces: 137
Number of polyhedrons that are tetrahedrons: 129
Mean Edge Ratio: 0.33616843588457035
Minimum Edge Ratio: 0.09759307171854792
Maximum Edge Ratio: 0.6564894374861545
Mean Tetrahedron per Polyhedron: 3.1322081575246132
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 12
Mean Faces per Polyhedron: 8.264416315049226
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 26

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 4.015403985977173 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 776
Number of barrier faces: 350
Number of polyhedra with barrier faces: 196
Number of polyhedrons that are tetrahedrons: 182
Mean Edge Ratio: 0.3529691116042896
Minimum Edge Ratio: 0.08437592784170189
Maximum Edge Ratio: 0.72143

##### n = 5000

In [19]:
n = 5000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns


Tiempo: 3.8896431922912598 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 711
Number of barrier faces: 198
Number of polyhedra with barrier faces: 137
Number of polyhedrons that are tetrahedrons: 129
Mean Edge Ratio: 0.33616843588457035
Minimum Edge Ratio: 0.09759307171854792
Maximum Edge Ratio: 0.6564894374861545
Mean Tetrahedron per Polyhedron: 3.1322081575246132
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 12
Mean Faces per Polyhedron: 8.264416315049226
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 26

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 4.014475345611572 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 776
Number of barrier faces: 350
Number of polyhedra with barrier faces: 196
Number of polyhedrons that are tetrahedrons: 182
Mean Edge Ratio: 0.3529691116042896
Minimum Edge Ratio: 0.08437592784170189
Maximum Edge Ratio: 0.7214

#### n = 10000

In [20]:
n = 10000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns


Tiempo: 3.848506212234497 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 711
Number of barrier faces: 198
Number of polyhedra with barrier faces: 137
Number of polyhedrons that are tetrahedrons: 129
Mean Edge Ratio: 0.33616843588457035
Minimum Edge Ratio: 0.09759307171854792
Maximum Edge Ratio: 0.6564894374861545
Mean Tetrahedron per Polyhedron: 3.1322081575246132
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 12
Mean Faces per Polyhedron: 8.264416315049226
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 26

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 4.016386985778809 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 776
Number of barrier faces: 350
Number of polyhedra with barrier faces: 196
Number of polyhedrons that are tetrahedrons: 182
Mean Edge Ratio: 0.3529691116042896
Minimum Edge Ratio: 0.08437592784170189
Maximum Edge Ratio: 0.72143